In [ ]:
import os
import shutil
import opendatasets as od
import pandas as pd
from IPython.display import display
from database.sql_connector import SqlConnector

In [ ]:
sql_con = SqlConnector()

# Indian IMDB Data

In [ ]:
kaggle_user = "saikrishnadupp"
folder_name = "top3000-imdb-indian-movies"

In [ ]:
od.download(f"https://www.kaggle.com/{kaggle_user}/{folder_name}", force=True)

In [ ]:
def process_director(val):
    val = val.replace('s:','').replace("'",'').replace('"','').replace('Director:','').lstrip().rstrip().split(',')
    val = [ind.lstrip().rstrip() for ind in val if len(ind.lstrip().rstrip())>2]
    return ', '.join(val)

def process_genre(val):
    val = val.lstrip().rstrip().split(',')
    val = [ind.lstrip().rstrip() for ind in val if len(ind.lstrip().rstrip())>2]
    return ', '.join(val)

In [ ]:
file = os.listdir(folder_name)[0]
df1 = pd.read_csv(os.path.join(folder_name,file), encoding="ISO-8859-1")
df1.drop(columns=['Age_of_content', 'Plot_summary', 'Cast_stars'], inplace=True)
df1.rename(columns={'Crew_dir':'Director', 'ImDBRating':'IMDB_Rating', 'Votes':'Number_Of_Votes', 'Date_of_Release':'Year_Of_Release'}, inplace=True)
df1 = df1[df1.Director.notna()].reset_index(drop=True)
df1 = df1[~(df1.Genre == '****')].reset_index(drop=True)
df1.Director =  df1.Director.apply(lambda x: process_director(x))
df1.Genre = df1.Genre.apply(lambda x: process_genre(x))
# df1 = df1.explode('Director').explode('Genre')
df1.head()

# Remove file 

In [ ]:
shutil.rmtree(folder_name)

# Ingest data 

In [ ]:
sql_con.run_query("""DELETE FROM KAGGLE.DATA.INDIAN_IMDB""")
sql_con.ingest_dataframe(df1, schema='DATA', table='INDIAN_IMDB')